# Data preparation

Run the following once! It just fetches additional data for geolocating IP addresses.

In [1]:
# !curl http://geolite.maxmind.com/download/geoip/database/GeoLiteCity.dat.gz > ./data/geoip.dat.gz
# !gunzip ./data/geoip.dat.gz

In [2]:
import pandas as pd
import numpy as np
import socket
from dns import reversename, resolver

import pygeoip
import pycountry_convert as pycountry

import plotly
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.io as pio

init_notebook_mode(connected=True)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

Let's define a function for IP resolution

In [3]:
rawdata = pygeoip.GeoIP('./data/geoip.dat')

def ipquery(ip):
    data = rawdata.record_by_name(ip)
    if data is not None:
        country = data['country_name']
        city = data['city']
        lon = data['longitude']
        lat = data['latitude']
        return [city, country, lat, lon]
    else:
        return [None, None, None, None]

In [4]:
def reverse_dns(ip):
    rev_name = reversename.from_address(ip)
    return str(resolver.query(rev_name,"PTR")[0])

def reverse_dns2(ip):
    try:
        return str(socket.gethostbyaddr(ip)[0])
    except:
        return 'unknown'

Loading dataframes

In [5]:
lst = pd.read_json('./data/log_topic_request.json').loc[2]['data']
topics_req_df = pd.DataFrame(lst)
topics_req_df.head(n=5)

,exists,format,id,time,topic,user
0,1,html,1,2018-07-20 12:53:30,computer science,angelo.salatino@open.ac.uk
1,1,html,2,2018-07-20 13:01:17,evaluation index system,5.45.207.68
2,1,html,3,2018-07-20 13:15:16,alpha phellandrene,66.249.79.98
3,1,html,4,2018-07-20 13:41:17,computer science,195.128.10.106
4,1,html,5,2018-07-20 13:44:44,microprocessor chips,195.128.10.106


In [6]:
topics_req_df.shape[0]

71432

In [7]:
lst = pd.read_json('./data/log_download_request.json').loc[2]['data']
downloads_req_df = pd.DataFrame(lst)
downloads_req_df.head(n=5)

,format,id,time,user,version
0,nt,1,2018-10-29 18:02:57,angelo.salatino@open.ac.uk,3.0
1,owl,2,2018-10-29 18:03:00,angelo.salatino@open.ac.uk,3.0
2,csv,3,2018-10-29 18:03:01,angelo.salatino@open.ac.uk,3.0
3,csv,4,2018-10-29 18:03:26,137.108.66.104,3.0
4,owl,5,2018-10-29 18:03:29,137.108.66.104,3.0


In [8]:
downloads_req_df.shape[0]

55

We have to patch email addresses (i.e. _user_ column); let's make them point to relevant IP address 
(these users are mainly us from KMi (**137.108.113.93**) or from Germany (Alex))

In [9]:
topics_req_df.loc[topics_req_df.user.str.contains('@')]['user'].unique()

array(['angelo.salatino@open.ac.uk', 'cso.skm3@gmail.com',
       'aliaksandr.birukou@springer.com', 'enrico.motta@open.ac.uk',
       'nina4ever27@gmail.com', 'rakeshgoad1999piplai@gmail.com',
       'danilodessi92@gmail.com', 'francesco.osborne@open.ac.uk',
       'martin.hlosta@open.ac.uk', 'matteo.cancellieri@open.ac.uk',
       'hakan.kiziloz@open.ac.uk', 'enrico.daga@open.ac.uk',
       'dasha.herrmannova@open.ac.uk'], dtype=object)

In [10]:
topics_req_df.loc[topics_req_df.user.str.contains('@open.ac.uk'), 'user'] = '137.108.200.90' # UK
topics_req_df.loc[topics_req_df.user.str.contains('aliaksandr.birukou'), 'user'] = '129.206.13.27' # DE
topics_req_df.loc[topics_req_df.user.str.contains('skm3'), 'user'] = '137.108.200.90'
topics_req_df.loc[topics_req_df.user.str.contains('danilodessi92@gmail.com'), 'user'] = '137.108.200.90'
topics_req_df.loc[topics_req_df.user.str.contains('nina4ever27'), 'user'] = '178.253.95.9' # Syria
topics_req_df.loc[topics_req_df.user.str.contains('rakeshgoad1999piplai'), 'user'] = '64.236.16.116' # US
topics_req_df.head()

,exists,format,id,time,topic,user
0,1,html,1,2018-07-20 12:53:30,computer science,137.108.200.90
1,1,html,2,2018-07-20 13:01:17,evaluation index system,5.45.207.68
2,1,html,3,2018-07-20 13:15:16,alpha phellandrene,66.249.79.98
3,1,html,4,2018-07-20 13:41:17,computer science,195.128.10.106
4,1,html,5,2018-07-20 13:44:44,microprocessor chips,195.128.10.106


In [11]:
downloads_req_df.loc[downloads_req_df.user.str.contains('@')]['user'].unique()

array(['angelo.salatino@open.ac.uk'], dtype=object)

In [12]:
downloads_req_df.loc[downloads_req_df.user.str.contains('@open.ac.uk'), 'user'] = '137.108.200.90'
downloads_req_df.head()

,format,id,time,user,version
0,nt,1,2018-10-29 18:02:57,137.108.200.90,3.0
1,owl,2,2018-10-29 18:03:00,137.108.200.90,3.0
2,csv,3,2018-10-29 18:03:01,137.108.200.90,3.0
3,csv,4,2018-10-29 18:03:26,137.108.66.104,3.0
4,owl,5,2018-10-29 18:03:29,137.108.66.104,3.0


Done!   
Now, let's create 4 new columns with the IP geolocation

In [13]:
ip_info = pd.concat([topics_req_df[['user']], downloads_req_df[['user']]]).drop_duplicates()
ip_info['host'] = ip_info['user'].map(lambda ip: reverse_dns2(ip))
ip_info[['city', 'country', 'lat', 'lon']] = ip_info.apply(lambda row: pd.Series(ipquery(row['user'])), axis=1)
ip_info.head()

,user,host,city,country,lat,lon
0,137.108.200.90,open-webservice-csvip.open.ac.uk,None,United Kingdom,51.4964,-0.1224
1,5.45.207.68,5-45-207-68.spider.yandex.com,None,Russian Federation,55.7386,37.6068
2,66.249.79.98,crawl-66-249-79-98.googlebot.com,Mountain View,United States,37.4192,-122.0574
3,195.128.10.106,unknown,None,Netherlands,52.3824,4.8995
7,66.249.79.103,crawl-66-249-79-103.googlebot.com,Mountain View,United States,37.4192,-122.0574


In [14]:
topics_req_df = pd.merge(topics_req_df, ip_info, left_on='user', right_on='user', how='left')
topics_req_df.head()

,exists,format,id,time,topic,user,host,city,country,lat,lon
0,1,html,1,2018-07-20 12:53:30,computer science,137.108.200.90,open-webservice-csvip.open.ac.uk,None,United Kingdom,51.4964,-0.1224
1,1,html,2,2018-07-20 13:01:17,evaluation index system,5.45.207.68,5-45-207-68.spider.yandex.com,None,Russian Federation,55.7386,37.6068
2,1,html,3,2018-07-20 13:15:16,alpha phellandrene,66.249.79.98,crawl-66-249-79-98.googlebot.com,Mountain View,United States,37.4192,-122.0574
3,1,html,4,2018-07-20 13:41:17,computer science,195.128.10.106,unknown,None,Netherlands,52.3824,4.8995
4,1,html,5,2018-07-20 13:44:44,microprocessor chips,195.128.10.106,unknown,None,Netherlands,52.3824,4.8995


In [15]:
topics_req_df.drop(topics_req_df[topics_req_df.host.str.contains('yandex')].index, inplace=True)
topics_req_df.drop(topics_req_df[topics_req_df.host.str.contains('google')].index, inplace=True)
topics_req_df.drop(topics_req_df[topics_req_df.host.str.contains('bot')].index, inplace=True)
topics_req_df.drop(topics_req_df[topics_req_df.host.str.contains('crawl')].index, inplace=True)
topics_req_df.drop(topics_req_df[topics_req_df.host.str.contains('yahoo')].index, inplace=True)

This leaves us with

In [16]:
topics_req_df.describe(include='all')

,exists,format,id,time,topic,user,host,city,country,lat,lon
count,5983,5983,5983,5983,5983,5983,5983,4358,5824,5824.000000,5824.000000
unique,2,5,5983,5233,3717,568,339,216,60,NaN,NaN
top,1,html,45618,2018-07-31 12:33:21,computer science,199.47.87.143,unknown,Alameda,United States,NaN,NaN
freq,5691,5921,1,4,374,1267,1403,2113,2448,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.080047,-41.757676
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.061223,70.423104
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-42.874700,-122.676200
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.773500,-122.278800
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.525700,-0.800000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.587550,-0.122400


In [17]:
downloads_req_df = pd.merge(downloads_req_df, ip_info, left_on='user', right_on='user', how='left')
downloads_req_df.head(n=5)

,format,id,time,user,version,host,city,country,lat,lon
0,nt,1,2018-10-29 18:02:57,137.108.200.90,3.0,open-webservice-csvip.open.ac.uk,None,United Kingdom,51.4964,-0.1224
1,owl,2,2018-10-29 18:03:00,137.108.200.90,3.0,open-webservice-csvip.open.ac.uk,None,United Kingdom,51.4964,-0.1224
2,csv,3,2018-10-29 18:03:01,137.108.200.90,3.0,open-webservice-csvip.open.ac.uk,None,United Kingdom,51.4964,-0.1224
3,csv,4,2018-10-29 18:03:26,137.108.66.104,3.0,unknown,None,United Kingdom,51.4964,-0.1224
4,owl,5,2018-10-29 18:03:29,137.108.66.104,3.0,unknown,None,United Kingdom,51.4964,-0.1224


In [18]:
downloads_req_df.describe(include='all')

,format,id,time,user,version,host,city,country,lat,lon
count,55,55,55,55,55,55,41,53,53.000000,53.000000
unique,3,55,55,34,2,19,19,12,NaN,NaN
top,owl,5,2018-12-12 11:50:46,137.108.66.104,3.0,unknown,Ho Chi Minh City,United States,NaN,NaN
freq,23,1,1,4,50,26,6,11,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.730396,20.403311
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.685377,74.305039
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-23.573300,-96.939800
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.375300,-46.641700
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.695700,6.633300
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.200000,78.474400


# Analysis of CSO downloads

In [19]:
versions = downloads_req_df.groupby('version')[['id']].nunique()

pie = go.Pie(values=versions.id,
              labels=versions.index,
              textposition='inside',
              name='Versions',
              hoverinfo='label+value+name',
              hole=.4)

donut = go.Layout(title='CSO downloads version fragmentation')
fig = go.Figure(data=[pie], layout=donut)
plotly.offline.iplot(fig)

In [20]:
formats = downloads_req_df.groupby('format')[['id']].nunique()

pie = go.Pie(values=formats.id,
              labels=formats.index,
              textposition='inside',
              name='Formats',
              hoverinfo='label+value+name',
              hole=.4)

donut = go.Layout(title='CSO downloads format fragmentation')
fig = go.Figure(data=[pie], layout=donut)
plotly.offline.iplot(fig)

In [21]:
data = downloads_req_df.groupby(['country'])['id'].count().reset_index()
tmp = [ dict(
        type = 'choropleth',
        locationmode = 'country names',
        locations = data['country'],
        z = data['id'],
        text = data['id'],
#         colorscale = [[0,"rgb(0,0,0)"]],
        autocolorscale = True,
        reversescale = False,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = True,
            title = '#downloads'),
      ) ]

layout = dict(
    title = 'Download distribution',
    geo = dict(
        showframe = False,
        showcoastlines = True,
        projection = dict(
            type = 'equirectangular'
        )
    )
)

fig = dict(data=tmp, layout=layout)
plotly.offline.iplot(fig, validate=False )

# Analysis of CSO topic requests

In [22]:
formats = topics_req_df.groupby('format')[['id']].nunique()

pie = go.Pie(values=formats.id,
              labels=formats.index,
              textposition='inside',
              name='Formats',
              hoverinfo='label+value+name',
              hole=.4)

donut = go.Layout(title='CSO topic requests format fragmentation')
fig = go.Figure(data=[pie], layout=donut)
plotly.offline.iplot(fig)

In [23]:
hits = topics_req_df.groupby('exists')[['id']].nunique()

pie = go.Pie(values=hits.id,
              labels=hits.index,
              textposition='inside',
              name='Hits',
              hoverinfo='label+value+name',
              hole=.4)

donut = go.Layout(title='CSO topics requests hits')
fig = go.Figure(data=[pie], layout=donut)
plotly.offline.iplot(fig)

In [24]:
data = topics_req_df.groupby(['country'])['user'].nunique().reset_index()

tmp = [ dict(
        type = 'choropleth',
        locationmode = 'country names',
        locations = data['country'],
        z = data['user'],
        text = data['user'],
#         colorscale = [[0,"rgb(0,0,0)"]],
        autocolorscale = True,
        reversescale = False,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = True,
            title = '#users'),
      ) ]

layout = dict(
    title = 'User distribution',
    geo = dict(
        showframe = False,
        showcoastlines = True,
        projection = dict(
            type = 'equirectangular'
        )
    )
)

fig = dict(data=tmp, layout=layout)
plotly.offline.iplot( fig, validate=False )

In [25]:
data = topics_req_df.groupby(['lat', 'lon'])['id'].count().reset_index()

bubbles = [dict(
    type = 'scattergeo',
    lon = data['lon'],
    lat = data['lat'],
    text = data['id'],
    marker = dict(
        size = data['id'],
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'
    ))]

layout = dict(
        title = 'Topic request distribution',
        showlegend = False,
        geo = dict(
            projection=dict(type='equiredtangular'),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict(data=bubbles, layout=layout)
plotly.offline.iplot(fig, validate=False)

In [26]:
bars_data = topics_req_df[topics_req_df.exists == '1'].groupby('topic')[['id']].count().sort_values(by='id', ascending=False).reset_index()
bars_data = bars_data[0:50]

trace0 = go.Bar(
    x = bars_data['topic'],
    y = bars_data['id'])

layout = go.Layout(title='Top-50 topics (hits)',
                  xaxis=dict(tickangle=-45,
                             automargin=True))
fig = go.Figure(data=[trace0], layout=layout)
plotly.offline.iplot(fig)


In [27]:
bars_data = topics_req_df[topics_req_df.exists == '0'].groupby('topic')[['id']].count().sort_values(by='id', ascending=False).reset_index()
bars_data = bars_data[0:50]

trace0 = go.Bar(
    x = bars_data['topic'],
    y = bars_data['id'],
    marker = dict(color='#cf0a5f'))

layout = go.Layout(title='Top-50 topics (misses)',
                  xaxis=dict(tickangle=-45,
                             automargin=True))
fig = go.Figure(data=[trace0], layout=layout)
plotly.offline.iplot(fig)


# Scrapbook

In [28]:
topics_req_df[topics_req_df.country == 'United States']

,exists,format,id,time,topic,user,host,city,country,lat,lon
508,1,html,509,2018-07-22 05:56:02,"1,2,3,6 tetrahydro 1 methyl 4 phenylpyridine",199.47.87.143,199-47-87-143.ip87.iparadigms.net,Alameda,United States,37.7735,-122.2788
509,1,html,510,2018-07-22 05:56:02,"1,3-dipolar cycloaddition",199.47.87.143,199-47-87-143.ip87.iparadigms.net,Alameda,United States,37.7735,-122.2788
510,1,html,511,2018-07-22 05:56:02,"1,2,3 triazole derivative",199.47.87.143,199-47-87-143.ip87.iparadigms.net,Alameda,United States,37.7735,-122.2788
511,1,html,512,2018-07-22 05:56:06,3 o methyldopamine,199.47.87.143,199-47-87-143.ip87.iparadigms.net,Alameda,United States,37.7735,-122.2788
512,1,html,513,2018-07-22 05:56:06,"2,3,4,5 tetrahydro 7,8 dihydroxy 1 phenyl 1h 3...",199.47.87.143,199-47-87-143.ip87.iparadigms.net,Alameda,United States,37.7735,-122.2788
513,1,html,514,2018-07-22 05:56:06,3 carene,199.47.87.143,199-47-87-143.ip87.iparadigms.net,Alameda,United States,37.7735,-122.2788
514,1,html,515,2018-07-22 05:56:10,3d display,199.47.87.143,199-47-87-143.ip87.iparadigms.net,Alameda,United States,37.7735,-122.2788
515,1,html,516,2018-07-22 05:56:10,32-nm node,199.47.87.143,199-47-87-143.ip87.iparadigms.net,Alameda,United States,37.7735,-122.2788
516,1,html,517,2018-07-22 05:56:10,"3,4 dihydroxyphenylacetic acid",199.47.87.143,199-47-87-143.ip87.iparadigms.net,Alameda,United States,37.7735,-122.2788
517,1,html,518,2018-07-22 05:56:13,4 aminobutyric acid,199.47.87.143,199-47-87-143.ip87.iparadigms.net,Alameda,United States,37.7735,-122.2788


In [29]:
topics_req_df[topics_req_df.host == 'unknown']

,exists,format,id,time,topic,user,host,city,country,lat,lon
3,1,html,4,2018-07-20 13:41:17,computer science,195.128.10.106,unknown,None,Netherlands,52.3824,4.8995
4,1,html,5,2018-07-20 13:44:44,microprocessor chips,195.128.10.106,unknown,None,Netherlands,52.3824,4.8995
5,1,html,6,2018-07-20 13:47:24,semantic web,195.128.10.106,unknown,None,Netherlands,52.3824,4.8995
6,1,html,7,2018-07-20 14:07:57,computer science,195.128.10.106,unknown,None,Netherlands,52.3824,4.8995
20,1,html,21,2018-07-20 16:50:11,classification process,185.84.51.218,unknown,None,Syrian Arab Republic,35.0000,38.0000
21,1,html,22,2018-07-20 16:51:33,classification accuracy,185.84.51.218,unknown,None,Syrian Arab Republic,35.0000,38.0000
22,1,html,23,2018-07-20 16:56:47,classification accuracy,185.84.51.218,unknown,None,Syrian Arab Republic,35.0000,38.0000
30,1,html,31,2018-07-20 18:10:27,computer science,172.21.81.253,unknown,None,None,NaN,NaN
31,1,html,32,2018-07-20 18:11:05,world wide web,172.21.81.253,unknown,None,None,NaN,NaN
32,1,html,33,2018-07-20 18:11:10,internet,172.21.81.253,unknown,None,None,NaN,NaN
